In [1]:
import pandas as pd
import subprocess

In [2]:
def generate_workspace():
    subprocess.run(["mkdir", "-p", "tmp_dir"])

In [3]:
import os


def cut_video(data:pd.Series, tmp_dir:str):
    """
    A function that cuts video according to pd.Series values.
    This function will execute following command using subprocess.
    ffmpeg -ss 00:01:00 -to 00:02:00  -i input.mp4 -c copy output.mp4
    
    :param data: a pandas.Series that represents the data of current video clip
    :param tmp_dir: a string that reprsents tmp_dir
    """
    start_str = data['start']
    end_str = data['end']
    country_name = data['name']
    save_name = data['save_name']
    out_file_name = os.path.join(tmp_dir, save_name)
    subprocess.check_output([
        "ffmpeg", 
        "-ss", 
        start_str, 
        "-to",
        end_str, 
        "-i", 
        "../OriginalVideo.mp4",
        "-c", 
        "copy", 
        out_file_name, 
        "-y"], 
        stderr=subprocess.STDOUT)

In [4]:
def change_video_speed(video_name:str, speed:float):
    """
    A function that changes video speed into designated speed.
    This will generate a video with %03ds.mp4.
    Example, if 000.mp4 was processed, it will be saved as 000s.mp4.
    
    This function will execute following command using subprocess.
    ffmpeg -i Break0.mp4 -vf "setpts=(PTS-STARTPTS)/2.5" -af atempo=2.5 a.mp4

    :param video_name: a string that represents video's name to process.
    :param speed: a float value that represents how fast the video should go.
    """
    subprocess.check_output([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "setpts=(PTS-STARTPTS)/%f" % speed,
        "-af",
        "atempo=%f" %speed,
        video_name.replace(".mp4", "s.mp4"),
        "-y"],
        stderr=subprocess.STDOUT)

In [25]:
def add_caption(video_name:str, country_name:str, value:str, 
                font_dir:str, font_color:str, font_size:int,
                box_enabled:bool, box_opacity:float, box_color:str, box_padding:int,
                text_position:tuple):
    """
    A function that adds caption to the video.
    This will generate a video with %03df.mp4.
    Example, if 000s.mp4 was processed, it will be saved as 000f.mp4.
    
    This function will execute following command using subprocess.
    ffmpeg -i 000s.mp4 -vf "drawtext=fontfile=/path/to/font.ttf:text='Stack Overflow':fontcolor=white:fontsize=24:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-text_w)/2:y=(h-text_h)/2" -codec:a copy 000f.mp4

    :param video_name: a string that represents video's name to process.
    :param country_name: a string that represents country's name.
    :param value: a string that represents value of that country.
    :param font_dir: a string that represents directory that leads to the font that current caption will be using.
    :param font_color: a string that represents color Ex) white.
    :param box_enabled: a boolean that represents whether or not to use text boxes.
    :param box_opacity: a float that sets box's opacity: 0 means it is it is not transparent at all.
    :param box_color: a string that represents box's color.
    :param box_padding: a string that adds padding to the text box.
    :param text_position: a tuple of text that represents where to locate texts.
    """
    text = country_name + "  " + value
    print(text)
    text_position_x = ""
    text_position_y = ""
    
    # set text_position_x using three options
    if text_position[0] == "center":  # when x position was center
        text_position_x = "(w-text_w)/2"  
    elif text_position[0] == "left":  # when x position was left
        text_position_x = "(%d)" % box_padding      
    elif text_position[0] == "right":  # when x position was right
        text_position_x = "(w-text_w-%d)" % box_padding
    else:  # if value was other than this, raise ValueError
        raise ValueError
    
    # set text_position_y using three options
    if text_position[1] == "center":  # when y position was center
        text_position_y = "(h-text_h)/2"
    elif text_position[1] == "up":  # when y position was up
        text_position_y = "(%d)" % box_padding
    elif text_position[1] == "down":  # when y position was down
        text_position_y = "(h-text_h-%d)" % box_padding
    else:  # if value was other than this, raise ValueError
        raise ValueError
    
    subprocess.check_output([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "drawtext=fontfile=/%s:text='%s':fontcolor=%s:fontsize=%d:box=%d:boxcolor=%s@%f:boxborderw=5:x=%s:y=%s" 
        % (font_dir, text, font_color, font_size, box_enabled, box_color, box_opacity, text_position_x, text_position_y),
        "-codec:a",
        "copy",
        video_name.replace("s.mp4", "f.mp4"),
        "-y"],
        stderr=subprocess.STDOUT)

In [26]:
if __name__ == "__main__":
    generate_workspace()
    tmp_dir = os.path.join(os.getcwd(), "tmp_dir")
    audio_dir = os.path.join(tmp_dir, "audio")
    data = pd.read_csv("../time_data.csv")
    cut_video(data.loc[0], tmp_dir)
    change_video_speed(os.path.join(tmp_dir, data.loc[0]['save_name']), 2)

In [27]:
from IPython.display import Video

Video("./tmp_dir/000s.mp4")

In [28]:
add_caption("./tmp_dir/000s.mp4", "break", "109999", "../fonts/comic.ttf", "white", 40, int(True), 0.5, "black", 10, ("left","down"))

break  109999


In [29]:
Video("./tmp_dir/000f.mp4")